In [3]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [13]:
df = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

df = df.with_columns(pl.col("DIAGNOSTICO1").str.replace(".", "", literal=True).str.pad_end(4, "X"))

In [5]:
ranking_egresos_nacionales = (
    df.group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"]))
    .agg([pl.len().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])
    .sort(by=["ANIO_EGRESO", "DIAGNOSTICO1", "n_egresos"], descending=True)
    .with_columns(
        pl.col("n_egresos")
        .rank(method="min", descending=True)
        .over(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .alias("ranking_egresos")
    )
)

In [7]:
ranking_egresos_nacionales.select(pl.col("DIAGNOSTICO1").str.replace(".", "", literal=True))

DIAGNOSTICO1
str
"""DESCONOCIDO"""
"""DESCONOCIDO"""
"""Z988"""
"""Z978"""
"""Z968"""
…
"""A010"""
"""A010"""
"""A010"""
